In [1]:
import os
import requests
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
API_BASE_URL = "https://www.aparat.com/api/fa/v1"
MAIN_PATH = "E:/Video_DL"

In [3]:
def get_playlist_info(playlist_id):
    playlist_url = f"{API_BASE_URL}/video/playlist/one/playlist_id/{playlist_id}"
    response = requests.get(playlist_url)
    if response.status_code !=  200:
        raise Exception(f"Failed to fetch playlist data: {response.status_code}")
    return response.json()

def get_video_info(video_hash):
    video_url = f"{API_BASE_URL}/video/video/show/videohash/{video_hash}"
    response = requests.get(video_url)
    if response.status_code !=  200:
        raise Exception(f"Failed to fetch video info for {video_hash}")
    return response.json()

def download_video(video_url, output_path):
    with requests.get(video_url, stream=True) as response:
        if response.status_code !=  200:
            raise Exception(f"Failed to download the video from {video_url}")
        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
    print(f'Downloaded to {output_path}')

def create_playlist_directory(playlist_title, main_path=MAIN_PATH):
    playlist_path = os.path.join(main_path, playlist_title)
    os.makedirs(playlist_path, exist_ok=True)
    return playlist_path

def save_video_links(playlist_path, video_urls):
    with open(os.path.join(playlist_path, "_URLS.txt"), "a") as f:
        for url in video_urls:
            f.write(url + "\n")

def download_playlist(playlist_id, playlist_start=1, only_link=False, main_path=MAIN_PATH):
    playlist_data = get_playlist_info(playlist_id)
    play_list_title = playlist_data['data']['attributes']['title']
    playlist_path = create_playlist_directory(play_list_title, main_path)

    videos = playlist_data['included']
    videos_df = pd.json_normalize(videos)
    videos_df = videos_df[videos_df['type'] == 'Video']

    video_urls = []
    for index, row in tqdm(videos_df.iterrows(), total=len(videos_df), desc="Processing videos"):
        if index + 1 >= playlist_start:
            video_id = row['attributes.uid']
            video_info = get_video_info(video_id)
            video_download_link_all = video_info['data']['attributes']['file_link_all']
            video_df = pd.DataFrame(video_download_link_all)

            try:
                max_quality = video_df['profile'].idxmax()
                download_url = video_df.loc[max_quality, 'urls'][0]
                video_urls.append(download_url)

                if not only_link:
                    quality = video_df.loc[max_quality, 'profile']
                    output_path = os.path.join(playlist_path, f"{index+1:02d}_{row['attributes.title']}-{quality}.mp4")
                    download_video(download_url, output_path)

            except (KeyError, IndexError) as e:
                print(f"Error processing video {video_id}: {str(e)}")

    save_video_links(playlist_path, video_urls)

In [ ]:
playlist_id =  000000
download_playlist(playlist_id)